In [1]:
# Import required libraries
import pandas as pd
from pathlib import Path

In [2]:
trading_df = pd.read_csv(
    Path("trailrun.csv"), 
    index_col="time", 
    infer_datetime_format=True, 
    parse_dates=True
)

In [3]:
# Review the DataFrame
trading_df.head()

,Index Level,close,"Cash Distribution (Cents-Per-Unit, AUD)",volume
time,,,,
2022-07-12,--,47.27,--,"5,826,388.00"
2022-07-11,319.28,47.75,--,"5,834,688.00"
2022-07-08,324.18,48.95,--,"5,834,688.00"
2022-07-07,324.78,49.84,--,"5,842,988.00"
2022-07-06,327.32,49.23,--,"5,842,988.00"


In [4]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["close"].pct_change()

# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

,Index Level,close,"Cash Distribution (Cents-Per-Unit, AUD)",volume,actual_returns
time,,,,,
2022-07-11,319.28,47.75,--,"5,834,688.00",0.010154
2022-07-08,324.18,48.95,--,"5,834,688.00",0.025131
2022-07-07,324.78,49.84,--,"5,842,988.00",0.018182
2022-07-06,327.32,49.23,--,"5,842,988.00",-0.012239
2022-07-05,330.96,49.86,--,"5,372,783.00",0.012797


,Index Level,close,"Cash Distribution (Cents-Per-Unit, AUD)",volume,actual_returns
time,,,,,
2004-10-11,99.72,24.60,--,"1,050,000.00",0.009438
2004-10-08,98.47,24.31,--,"600,000.00",-0.011789
2004-10-07,100.51,24.82,--,"600,000.00",0.020979
2004-10-06,99.49,24.56,--,"600,000.00",-0.010475
2004-10-05,100,24.69,--,"600,000.00",0.005293


In [5]:
# Define a window size of 4
short_window = 4

# Create a simple moving average (SMA) using the short_window and assign this to a new columns called sma_fast
trading_df["sma_fast"] = trading_df["close"].rolling(window=short_window).mean()

In [6]:
# Define a window size of 100
long_window = 100

# Create a simple moving average (SMA) using the long_window and assign this to a new columns called sma_slow
trading_df["sma_slow"] = trading_df["close"].rolling(window=long_window).mean()

In [7]:
# Drop the NaNs using dropna()
trading_df = trading_df.dropna()

In [8]:
# Assign a copy of the sma_fast and sma_slow columns to a new DataFrame called X
X = trading_df[["sma_fast", "sma_slow"]].copy()

# Display sample data
display(X.head())
display(X.tail())

,sma_fast,sma_slow
time,,
2022-02-16,51.9150,44.8113
2022-02-15,52.4500,44.8640
2022-02-14,52.7600,44.8984
2022-02-11,52.7525,44.9246
2022-02-10,52.6975,44.9615


,sma_fast,sma_slow
time,,
2004-10-11,24.1700,23.4297
2004-10-08,24.3800,23.4312
2004-10-07,24.5250,23.4418
2004-10-06,24.5725,23.4507
2004-10-05,24.5950,23.4622


In [9]:
# Create a new column in the trading_df called signal setting its value to zero.
trading_df["signal"] = 0.0

In [10]:
# Create the signal to buy
trading_df.loc[(trading_df["actual_returns"] >= 0), "signal"] = 1

In [11]:
# Create the signal to sell
trading_df.loc[(trading_df["actual_returns"] < 0), "signal"] = -1

In [12]:
# Copy the new signal column to a new Series called y.
y = trading_df["signal"].copy()

In [13]:
#TEST&SPLIT

In [14]:
# Imports 
from pandas.tseries.offsets import DateOffset

In [15]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2004-10-05 00:00:00


In [16]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

2005-01-05 00:00:00


In [22]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data
# X_train.head()


SyntaxError: invalid syntax (661295077.py, line 4)

In [18]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Display sample data
X_test.head()

,sma_fast,sma_slow
time,,
2005-01-05,22.8025,23.5032
2005-01-04,22.9600,23.5006
2005-01-03,23.1725,23.4996
2004-12-31,23.3125,23.4930
2004-12-30,23.4550,23.4889


In [19]:
#STANDARDIZE
# Imports
from sklearn.preprocessing import StandardScaler

In [20]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Display the sample predictions
training_signal_predictions[:10]

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by StandardScaler.

In [ ]:
# Imports
from sklearn import svm
from sklearn.metrics import classification_report

In [ ]:
# Create the classifier model
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

In [ ]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)
print(training_report)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
svm_testing_report = classification_report(y_test, testing_signal_predictions)
print(svm_testing_report)

In [ ]:
# Create a new empty predictions DataFrame using code provided below.
predictions_df = pd.DataFrame(index=X_test.index)
predictions_df["predicted_signal"] = testing_signal_predictions
predictions_df["actual_returns"] = trading_df["actual_returns"]
predictions_df["trading_algorithm_returns"] = predictions_df["actual_returns"] * predictions_df["predicted_signal"]
predictions_df.head()

In [ ]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()

In [ ]:
#LOGISTICREGRESSION
# Import LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression

In [ ]:
# Create an instance of the LogisticRegression model
logistic_regression_model = LogisticRegression()

In [ ]:
# Fit the LogisticRegression model
logistic_regression_model.fit(X_train_scaled, y_train)

In [ ]:
# Use the trained LogisticRegression model to predict the trading signals for the training data
lr_training_signal_predictions = logistic_regression_model.predict(X_train_scaled)

# Display the predictions
lr_training_signal_predictions

In [ ]:
# Generate a classification report using the training data and the logistic regression model's predications
lr_training_report = classification_report(y_train, lr_training_signal_predictions)

# Review the classification report
print(lr_training_report)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
lr_testing_signal_predictions = logistic_regression_model.predict(X_test_scaled)

In [ ]:
# Generate a classification report using the testing data and the logistic regression model's predictions
lr_testing_report = classification_report(y_test, lr_testing_signal_predictions)

# Review the testing classification report
print(lr_testing_report)

In [ ]:
# Print the classification report for the Logistic Regression model using the test data
print("Logistic Regression Classification Report")
print(lr_testing_report)

In [ ]:
# Print the classification report for the SVM model using the test data
print("SVM Classification Report")
print(svm_testing_report)